In [1]:
import os
import glob
import pandas as pd

In [8]:
submissions_dict = []
for file in glob.glob('submissions/*.csv'):
    df = pd.read_csv(file, names=['id', 'label'])
    submissions_dict.append({os.path.basename(file).split('.')[0].split('-')[0]: df})
submissions_dict.sort(key=lambda x: list(x.keys())[0])

wavs = os.listdir('data/TIL_test_normalized')

In [9]:
wav_preds_dict = {}
for wav in wavs:
    wav_preds_dict[wav] = {}
    for submission in submissions_dict:
        score = list(submission.keys())[0]
        df = submission[score]
        pred = df[df['id'] == wav]['label'].values[0]
        try:
            wav_preds_dict[wav][pred] += [score]
        except KeyError:
            wav_preds_dict[wav][pred] = [score]

In [10]:
wav_preds_dict_scored = {}
for wav, preds in wav_preds_dict.items():
    wav_preds_dict_scored[wav] = {k: sum(map(int, v)) / len(submissions_dict) for k, v in preds.items()}

wav_preds_dict_max_score = {}
for wav, preds in wav_preds_dict_scored.items():
    wav_preds_dict_max_score[wav] = max(preds, key=preds.get)

In [12]:
ans_df = pd.DataFrame.from_dict(wav_preds_dict_max_score, orient='index')
ans_df.to_csv('combined_without.csv', header=False)

In [16]:
c = 0
for wav, preds in wav_preds_dict.items():
    if len(preds) == 5:
        print(wav)
        c += 1

c

0511fae35f.wav
0a79bd9dc0.wav
123e60142b.wav
15b3b2d2b0.wav
1888317cd7.wav
1a3ba17b8b.wav
2031a15187.wav
2829825223.wav
29ebb4e355.wav
2dba08d924.wav
3a816b0d50.wav
54573f0f81.wav
5a3e04eb19.wav
75dbedaeab.wav
877c664143.wav
8ff6a50fbb.wav
9d1263b826.wav
9efef8fe7e.wav
a4189a8711.wav
a5e6cb0912.wav
b9c6038815.wav
c20419c236.wav
c6aa7edc1d.wav
d418ef05e5.wav
e08ba27aa9.wav
e7e6074ba6.wav
f0c2a69ca5.wav


27

In [11]:
ans_df_new = pd.read_csv('answer-new.csv')
not_sure = ans_df_new[ans_df_new['notsure'] == 1]
notsure_dict = {}
for wav, preds in wav_preds_dict_max_score.items():
    if wav in not_sure['filename'].values:
        notsure_dict[wav] = preds

In [12]:
# update ans_df_new with notsure dict
for wav, preds in notsure_dict.items():
    ans_df_new.loc[ans_df_new['filename'] == wav, 'handlabelled'] = preds

ans_df_new.to_csv('combined-new2.csv', index=False, header=False, columns=['filename', 'handlabelled'])